# Sparse Representation Classification (block)


In [ ]:
import torch
from matplotlib import pyplot as plt

import dataset
from src_block import src_block_eval

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

In [ ]:
train_ds = dataset.train()
train_ds = torch.tensor(train_ds, device=device)
train_ds.shape

In [ ]:
test_ds = dataset.test()
test_ds = torch.tensor(test_ds, device=device)
test_ds.shape

In [ ]:
def split4(x: torch.Tensor):
    row_p = [0, 15, 30]
    col_p = [0, 10, 26, 32, 40]
    res = []
    for j in range(len(col_p) - 1):
        for i in range(len(row_p) - 1):
            m = x[:, :, col_p[j]:col_p[j + 1], row_p[i]:row_p[i + 1]]
            res.append(m.contiguous())
    return res


train_ds = split4(train_ds)
test_ds = split4(test_ds)

In [ ]:
accu = src_block_eval(train_ds, test_ds, device=device)

In [ ]:
accu.mean()

In [ ]:
plt.bar(range(len(accu)), accu)
plt.savefig("src_block.png")
plt.show()